__Read in the VCF used for the genopop files from Michelle that were used for the parentage paper. 
Looking through the pinskylab/genomics repo, the genpop I used for parentage came from Michelle /local/home/katrinac/parentage/colony2/20190422_894loci/input/seq33-03_norecap.gen
She made this in https://raw.githubusercontent.com/pinskylab/genomics/master/Protocols/05_remove-regenos.md
She made it from a genepop file seq33_03_baits_only_SNPs.gen
Looks like this was converted to genepop format from FIL_6.recode.vcf.zip (https://github.com/pinskylab/genomics/blob/master/data/FIL_6.recode.vcf.zip) in the script https://raw.githubusercontent.com/pinskylab/genomics/master/Protocols/04_filtering_scheme-6-with-70-35.md
Do some quick QC to verify that this was the right source file once complete__

__First, get in the right working directory__

In [9]:
%cd ~/ClownfishGWAS/

/local/home/katrinac/ClownfishGWAS


__Copy the file from genomics directory, unzip the VCF, and then move into our data directory. Then look inside to make sure it looks right__

In [38]:
!cp -r ~/genomics/data/FIL_6.recode.vcf.zip ~/ClownfishGWAS/data/ #use -r because we are copying a directory, so needs recursive flag


In [40]:
!unzip data/FIL_6.recode.vcf.zip/FIL_6.recode.vcf.zip #I guess it was stored in a folder called the same thing as a file, no point fixing this it's unzipped 

Archive:  data/FIL_6.recode.vcf.zip/FIL_6.recode.vcf.zip
  inflating: FIL_6.recode.vcf        
replace __MACOSX/._FIL_6.recode.vcf? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [42]:
!mv FIL_6.recode.vcf data/


In [43]:
!head data/FIL_6.recode.vcf

##source=freeBayes v1.0.2-58-g054b257
##reference=reference.fasta
##contig=<ID=dDocent_Contig_269957,length=211>
##phasing=none
##filter="AB > 0.2 & AB < 0.8 | AB < 0.01 | AB > 0.99 genotypes filtered with: QR > 0 | QA > 0"
##filter="QUAL / DP > 0.2"
##INFO=<ID=NS,Number=1,Type=Integer,Description="Number of samples with data">
##INFO=<ID=DP,Number=1,Type=Integer,Description="Total read depth at the locus">
##INFO=<ID=DPB,Number=1,Type=Float,Description="Total read depth per bp at the locus; bases in reads overlapping / bases in haplotype">
##INFO=<ID=AC,Number=A,Type=Integer,Description="Total number of alternate alleles in called genotypes">


__Filter the VCF to only include insidividuals from the parentage analysis quality checked by Michelle, and to only include individuals on HECR or STME__

In [48]:
!vcftools --vcf data/FIL_6.recode.vcf --keep script_output/ForMolEcoGroupAnemSppLigIDs.txt --recode --recode-INFO-all --out script_output/ForMolEcoGroup #get sample names to make sure it's coded with ligation ID


VCFtools - 0.1.15
(C) Adam Auton and Anthony Marcketta 2009

Parameters as interpreted:
	--vcf data/FIL_6.recode.vcf
	--keep script_output/ForMolEcoGroupAnemSppLigIDs.txt
	--recode-INFO-all
	--out script_output/ForMolEcoGroup
	--recode

Keeping individuals in 'keep' list
After filtering, kept 1521 out of 2908 Individuals
Outputting VCF file...
After filtering, kept 1005 out of a possible 1005 Sites
Run Time = 6.00 seconds


__Need to add the VCF file tag to the VCF, because someone in past filtering that file tag was lost from the FIL_6.recode.vcf__

In [ ]:
!sed -i '1 i\##fileformat=VCFv4.1'  script_output/ForMolEcoGroup.recode.vcf

__Convert to plink for GWAS__

In [100]:
#!vcftools --vcf script_output/ForMolEcoGroup.recode.vcf --out script_output/ForMolEcoGroup.recode --plink #commented this out so it didn't take up the page

__Dangit we need to make our own map https://www.biostars.org/p/109690/__

In [61]:
#!bcftools view -H script_output/FIL_6.recodeTest.vcf | cut -f 1 | uniq | awk '{print $0"\t"$0}' > script_output/ForMolEcoGroup.chrom-map.txt

__Okay so now we have to fix a problem- the vcf doesn't have contigs names in the header and bcftools wants that, so we have to add it, following instructions here: https://www.biostars.org/p/109690/. First, generate an index file from the fasta file, which we'll use to make a header file. info on fasta.fai https://www.biostars.org/p/98885/__

In [34]:
!samtools faidx ~/genomics/data/Pinsky_contigs1050.fasta 


In [35]:
!head /local/home/katrinac/genomics/data/Pinsky_contigs1050.fasta.fai 

dDocent_Contig_11	211	19	211	212
dDocent_Contig_80	211	250	211	212
dDocent_Contig_85	211	481	211	212
dDocent_Contig_107	211	713	211	212
dDocent_Contig_165	211	945	211	212
dDocent_Contig_276	211	1177	211	212
dDocent_Contig_315	211	1409	211	212
dDocent_Contig_333	211	1641	211	212
dDocent_Contig_394	211	1873	211	212
dDocent_Contig_398	211	2105	211	212


__Now we need to use this fasta index file to create a header file to insert into the vcf. The printf () command tells it to print what's in the parentheses. Then, we're using the first column as contig names and the second contig length. Semicolon just lets us do 2 commands in a line.__

In [42]:
!awk '{printf("##contig=<ID=%s,length=%d>\n",$1,$2);}' /local/home/katrinac/genomics/data/Pinsky_contigs1050.fasta.fai > /local/home/katrinac/ClownfishGWAS/script_output/ToEditHeader.txt 

In [43]:
!head /local/home/katrinac/ClownfishGWAS/script_output/ToEditHeader.txt

##contig=<ID=dDocent_Contig_11,length=211>
##contig=<ID=dDocent_Contig_80,length=211>
##contig=<ID=dDocent_Contig_85,length=211>
##contig=<ID=dDocent_Contig_107,length=211>
##contig=<ID=dDocent_Contig_165,length=211>
##contig=<ID=dDocent_Contig_276,length=211>
##contig=<ID=dDocent_Contig_315,length=211>
##contig=<ID=dDocent_Contig_333,length=211>
##contig=<ID=dDocent_Contig_394,length=211>
##contig=<ID=dDocent_Contig_398,length=211>


In [62]:
#!awk '/^#CHROM/ { printf("##contig=<ID=dDocent_Contig_11,length=211>\n##contig=<ID=dDocent_Contig_80,length=211>\n");} {print;}' script_output/ForMolEcoGroup.recode.vcf > script_output/contigsInHeader.vcf
#awk '/^#CHROM/ { printf("##contig=<ID=1,length=195471971>\n##contig=<ID=2,length=182113224>\n");} {print;}' in.vcf > out.vcf

__Then we take this text doc of files and paste all of it into the VCF file at line 5__

In [63]:
!head script_output/ForMolEcoGroup.recode.vcf

##fileformat=VCFv4.1
##source=freeBayes v1.0.2-58-g054b257
##reference=reference.fasta
##contig=<ID=dDocent_Contig_11,length=211>
##contig=<ID=dDocent_Contig_80,length=211>
##contig=<ID=dDocent_Contig_85,length=211>
##contig=<ID=dDocent_Contig_107,length=211>
##contig=<ID=dDocent_Contig_165,length=211>
##contig=<ID=dDocent_Contig_276,length=211>
##contig=<ID=dDocent_Contig_315,length=211>


__Lastly, we annotate the VCF with bcftools to get the contigs in the header!__

In [66]:
!bcftools annotate --set-id +'%CHROM\_%POS' script_output/ForMolEcoGroup.recode.vcf > script_output/SnpsWithContigIDsForMolEcoGroup.vcf

__Now we use bcftools to make our own map file__

In [67]:
!bcftools view -H script_output/SnpsWithContigIDsForMolEcoGroup.vcf | cut -f 1 | uniq | awk '{print $0"\t"$0}' > script_output/ForMolEcoGroup.chrom-map.txt

__Now, we make a ped file with that map https://www.biostars.org/p/109690/__

In [90]:

!vcftools --vcf script_output/SnpsWithContigIDsForMolEcoGroup.vcf  --plink --chrom-map script_output/ForMolEcoGroup.chrom-map.txt --out script_output/ForMolEcoGroup


VCFtools - 0.1.15
(C) Adam Auton and Anthony Marcketta 2009

Parameters as interpreted:
	--vcf script_output/SnpsWithContigIDsForMolEcoGroup.vcf
	--chrom-map script_output/ForMolEcoGroup.chrom-map.txt
	--out script_output/ForMolEcoGroup
	--plink

After filtering, kept 1521 out of 1521 Individuals
Writing PLINK PED and MAP files ... 
	Read 485 chromosome mapping file entries.
	PLINK: Only outputting biallelic loci.
Done.
After filtering, kept 1005 out of a possible 1005 Sites
Run Time = 1.00 seconds


__I then went into R and use the table of fish IDs and corresponding host anem spp to fill in column 6 of the ped, which should house the phenotype__

__Filter minor allele frequency and missingness. Then I used a maximum pihat cutoff of 0.25 for the relatedness filtering, based on the logic here https://www.biostars.org/p/75335/
I quickly checked this against the Colony relatedness results, and the samples excluded by plink are related according to colony, I think this is fine for our purposes.__

In [96]:
 !plink --file script_output/ForMolEcoGroup.recode --allow-no-sex --make-bed --maf 0.02 --geno 0.1 --rel-cutoff 0.25 --out script_output/ForMolEcoGroup.maf0.2gen0.1NoRelatedness

PLINK v1.90b6.21 64-bit (19 Oct 2020)          www.cog-genomics.org/plink/1.9/
(C) 2005-2020 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to script_output/ForMolEcoGroup.maf0.2gen0.1NoRelatedness.log.
Options in effect:
  --allow-no-sex
  --file script_output/ForMolEcoGroup.recode
  --geno 0.1
  --maf 0.02
  --make-bed
  --out script_output/ForMolEcoGroup.maf0.2gen0.1NoRelatedness
  --rel-cutoff 0.25

515706 MB RAM detected; reserving 257853 MB for main workspace.
.ped scan complete (for binary autoconversion).
Performing single-pass .bed write (1003 variants, 1521 people).


--file: script_output/ForMolEcoGroup.maf0.2gen0.1NoRelatedness-temporary.bed +49505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394%
script_output/ForMolEcoGroup.maf0.2gen0.1NoRelatedness-temporary.bim +
script_output/ForMolEcoGroup.maf0.2gen0.1NoRelatedness-temporary.fam written.
1003 variants loaded from .bim file.
1521 people (0 males, 0 females, 1521 ambiguous) loaded from .fam.
Ambiguous sex IDs written to
script_output/ForMolEcoGroup.maf0.2gen0.1NoRelatedness.nosex .
1521 phenotype values loaded from .fam.
Using up to 71 threads (change this with --threads).
Before main variant filters, 1521 founders and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
Total genotyping rate is 0.930888.
110 variants removed due to missing genotype data (--geno).
0 var

__tested that association command works, and it does__